In [ ]:
def plot_experiment_results(results):
    """
    Plot the results from the grassroots vs. establishment experiment with publication-quality visualizations.
    
    Parameters:
    -----------
    results : dict
        Results dictionary from run_grassroots_vs_establishment_experiment()
    """
    network_types = list(results.keys())
    strategies = list(results[network_types[0]].keys())
    
    # Setting up a more professional color scheme
    colors = [SUPPORTER_COLOR, UNDECIDED_COLOR, OPPOSITION_COLOR]
    opinion_labels = ['Supporters', 'Undecided', 'Opposition']
    
    # Create figure for barplots of final opinion distributions
    fig, axes = plt.subplots(1, len(network_types), figsize=(16, 6))
    
    for i, network_type in enumerate(network_types):
        ax = axes[i]
        
        # Data for plotting
        data = {
            'Strategy': [],
            'Opinion': [],
            'Proportion': []
        }
        
        for strategy in strategies:
            for opinion, idx in [('Supporters', SUPPORTER), ('Undecided', UNDECIDED), ('Opposition', OPPOSITION)]:
                # Fix: Use correct keys for each opinion
                opinion_key = 'supporter_final' if opinion == 'Supporters' else ('undecided_final' if opinion == 'Undecided' else 'opposition_final')
                values = results[network_type][strategy][opinion_key]
                mean_value = np.mean(values)
                
                # Add to data for seaborn plotting
                data['Strategy'].extend([strategy] * len(values))
                data['Opinion'].extend([opinion] * len(values))
                data['Proportion'].extend(values)
        
        # Plot with seaborn
        sns.barplot(x='Strategy', y='Proportion', hue='Opinion', 
                   data=data, ax=ax, 
                   palette=colors)
        
        # Enhanced styling
        network_type_titles = {
            'scale-free': 'Scale-Free Network\n(Urban Centers)',
            'small-world': 'Small-World Network\n(Suburban Areas)',
            'random': 'Random Network\n(Rural Communities)'
        }
        
        ax.set_title(network_type_titles.get(network_type, network_type.title()), fontsize=14, pad=10)
        ax.set_ylim(0, 1)
        ax.set_xlabel('')
        
        if i == 0:
            ax.set_ylabel('Proportion of Population', fontsize=12)
        else:
            ax.set_ylabel('')
        
        if i == len(network_types) - 1:
            ax.legend(title="Opinion", title_fontsize=12, fontsize=10, bbox_to_anchor=(1.05, 0.5), loc='center left')
        else:
            ax.legend([], [], frameon=False)
        
        # Enhanced x-axis labels
        strategy_labels = {
            'No shock': 'No Intervention',
            'Establishment (High-degree targets)': 'Establishment\nStrategy',
            'Grassroots (Random targets)': 'Grassroots\nStrategy'
        }
        ax.set_xticklabels([strategy_labels.get(s, s) for s in strategies])
        plt.setp(ax.get_xticklabels(), rotation=0, ha='center')
        
        # Add subtle grid
        ax.grid(axis='y', alpha=0.3)
        
        # Add data value labels on the bars
        for p in ax.patches:
            ax.annotate(f'{p.get_height():.2f}', 
                       (p.get_x() + p.get_width() / 2., p.get_height()), 
                       ha='center', va='bottom', fontsize=8, color='black', 
                       xytext=(0, 1), textcoords='offset points')
    
    # Overall title
    plt.suptitle('Opinion Distribution Across Network Types and Intervention Strategies', 
                y=0.98, fontsize=16)

    
    plt.tight_layout()
    fig.subplots_adjust(top=0.85, bottom=0.15)
    
    # Create a second figure for opinion evolution over time
    fig2, axes2 = plt.subplots(len(network_types), len(strategies), figsize=(16, 12))
    
    # Add shock period indicator
    shock_start = 10
    shock_end = 10 + 20  # shock_duration
    
    for i, network_type in enumerate(network_types):
        for j, strategy in enumerate(strategies):
            ax = axes2[i, j]
            
            if 'history' in results[network_type][strategy]:
                history = results[network_type][strategy]['history']
                
                supporters = [h[SUPPORTER] for h in history]
                undecided = [h[UNDECIDED] for h in history]
                opposition = [h[OPPOSITION] for h in history]
                
                steps = range(len(history))
                
                # Plot with enhanced styling
                ax.plot(steps, supporters, '-', color=SUPPORTER_COLOR, linewidth=2.5, label='Supporters')
                ax.plot(steps, undecided, '-', color=UNDECIDED_COLOR, linewidth=2.5, label='Undecided')
                ax.plot(steps, opposition, '-', color=OPPOSITION_COLOR, linewidth=2.5, label='Opposition')
                
                # Highlight the shock period with better styling
                ax.axvspan(shock_start, shock_end, alpha=0.15, color='gray', edgecolor='none')
                
                # Add a vertical line at the shock start and end with labels
                if i == 0:  # Only add text labels on top row
                    ax.axvline(x=shock_start, color='gray', linestyle='--', alpha=0.7)
                    ax.axvline(x=shock_end, color='gray', linestyle='--', alpha=0.7)
                    ax.text(shock_start - 5, 0.95, 'Intervention\nStart', ha='right', va='top', 
                           color='black', fontsize=8, bbox=dict(facecolor='white', alpha=0.7, pad=2))
                    ax.text(shock_end + 5, 0.95, 'Intervention\nEnd', ha='left', va='top', 
                           color='black', fontsize=8, bbox=dict(facecolor='white', alpha=0.7, pad=2))
                
                ax.set_ylim(0, 1)
                ax.grid(True, alpha=0.3)
                
                # Add titles
                if i == 0:
                    ax.set_title(strategy_labels.get(strategy, strategy), fontsize=14, pad=10)
                
                if j == 0:
                    ax.set_ylabel(f'{network_type_titles.get(network_type, network_type.title())}\nProportion', fontsize=12)
                
                # Add legend only on first plot
                if i == 0 and j == 0:
                    ax.legend(fontsize=10, loc='upper right')
                
                # Add x-label only on bottom row
                if i == len(network_types) - 1:
                    ax.set_xlabel('Time Step', fontsize=12)
                
                # Enhance tick labels
                ax.tick_params(axis='both', which='major', labelsize=10)
                
                # Add annotations for final proportions
                final_s = supporters[-1]
                final_u = undecided[-1]
                final_o = opposition[-1]
                
                # Add final values annotation
                ax.annotate(f'Final: {final_s:.2f}', xy=(len(steps)-10, final_s), 
                           xytext=(5, 0), textcoords='offset points', 
                           color=SUPPORTER_COLOR, fontsize=8, ha='left', va='center')
                
                ax.annotate(f'Final: {final_o:.2f}', xy=(len(steps)-10, final_o), 
                           xytext=(5, 0), textcoords='offset points', 
                           color=OPPOSITION_COLOR, fontsize=8, ha='left', va='center')
    
    # Add a comprehensive title
    plt.suptitle('Opinion Evolution Over Time by Network Type and Intervention Strategy',
                y=0.98, fontsize=16)
    
    plt.tight_layout()
    fig2.subplots_adjust(top=0.92, hspace=0.3, wspace=0.3)
    
    return fig, fig2

def plot_battleground_results(results, save_path="figures/battleground_results.pdf"):
    """
    Plot the results from the network battleground experiment with enhanced visualizations.
    
    Parameters:
    -----------
    results : dict
        Results dictionary from run_network_battleground_experiment()
    save_path : str
        Path to save the figure
        
    Returns:
    --------
    matplotlib.figure.Figure
        Figure containing the plots
    """
    network_types = list(results.keys())
    
    # Create figure with GridSpec for more flexible layout
    fig = plt.figure(figsize=(16, 8))
    
    # Create a simple 1x2 grid instead of 2x2
    gs = GridSpec(1, 2, figure=fig)
    
    # Left: Supporter gain bar chart
    ax1 = fig.add_subplot(gs[0, 0])
    
    # Format data for plotting
    gain_means = [np.mean(results[nt]['supporter_gain']) for nt in network_types]
    gain_std = [np.std(results[nt]['supporter_gain']) for nt in network_types]
    
    # Enhanced bar colors - use a blue gradient for supporter gain
    bar_colors = ['#6baed6', '#4292c6', '#2171b5']
    
    # Plot bars with error
    bars = ax1.bar(range(len(network_types)), gain_means, yerr=gain_std, 
                  capsize=10, color=bar_colors, edgecolor='black', linewidth=1)
    
    # Add data labels on top of bars
    for i, bar in enumerate(bars):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + gain_std[i] + 0.01,
                f'{gain_means[i]:.3f}', ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    # Enhanced styling
    ax1.set_title('Average Supporter Gain by Network Type', fontsize=16, pad=15)
    ax1.set_ylabel('Net Supporter Gain', fontsize=14)
    ax1.set_xticks(range(len(network_types)))
    
    # Better x-axis labels with descriptions
    network_labels = {
        'urban_center': 'Urban Centers\n(Scale-Free)',
        'suburban_area': 'Suburban Areas\n(Small-World)',
        'rural_community': 'Rural Communities\n(Random)'
    }
    ax1.set_xticklabels([network_labels.get(nt, nt) for nt in network_types], fontsize=12)
    
    # Add subtle grid
    ax1.grid(axis='y', alpha=0.3, linestyle='--')
    ax1.set_axisbelow(True)
    
    # Right: Campaign efficiency bar chart
    ax2 = fig.add_subplot(gs[0, 1])
    
    # Format data for plotting
    efficiency_means = [np.mean(results[nt]['resource_efficiency']) for nt in network_types]
    efficiency_std = [np.std(results[nt]['resource_efficiency']) for nt in network_types]
    
    # Enhanced bar colors - use a green gradient for efficiency
    efficiency_colors = ['#74c476', '#41ab5d', '#238b45']
    
    # Plot bars with error
    bars = ax2.bar(range(len(network_types)), efficiency_means, yerr=efficiency_std, 
                  capsize=10, color=efficiency_colors, edgecolor='black', linewidth=1)
    
    # Add data labels on top of bars
    for i, bar in enumerate(bars):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + efficiency_std[i] + 0.2,
                f'{efficiency_means[i]:.2f}', ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    # Enhanced styling
    ax2.set_title('Campaign Efficiency by Network Type', fontsize=16, pad=15)
    ax2.set_ylabel('Gain per Resource Unit', fontsize=14)
    ax2.set_xticks(range(len(network_types)))
    ax2.set_xticklabels([network_labels.get(nt, nt) for nt in network_types], fontsize=12)
    
    # Add subtle grid
    ax2.grid(axis='y', alpha=0.3, linestyle='--')
    ax2.set_axisbelow(True)
    
    plt.tight_layout()
    plt.suptitle('Strategic Resource Allocation for Opinion Campaigns', fontsize=18, y=0.98)
    fig.subplots_adjust(top=0.85)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    # Save figure
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    print(f"Figure saved to {save_path}")
    
    return fig

def plot_timing_results(results, save_path="figures/timing_experiment.pdf"):
    """
    Plot the results from the timing experiment with publication-quality visualizations
    including confidence intervals and proper error bars.
    
    Parameters:
    -----------
    results : dict
        Results dictionary from run_timing_experiment()
    save_path : str
        Path to save the figure
        
    Returns:
    --------
    matplotlib.figure.Figure
        Figure containing the plots
    """
    # Set seaborn style for publication quality
    import seaborn as sns
    sns.set_theme(style="whitegrid", context="paper")
    
    timing_labels = list(results.keys())
    
    # Create a more comprehensive figure with GridSpec
    fig = plt.figure(figsize=(12, 10))
    gs = GridSpec(2, 2, figure=fig, height_ratios=[2, 1])
    
    # Top: Time evolution with confidence intervals
    ax1 = fig.add_subplot(gs[0, :])
    
    # Define timing-specific colors and styles
    colors = {'Early Campaign': '#1f77b4', 'Late Campaign': '#d62728'}
    
    for i, timing in enumerate(timing_labels):
        if 'all_histories' in results[timing] and len(results[timing]['all_histories']) > 0:
            # Calculate confidence intervals from all trials
            num_steps = len(results[timing]['all_histories'][0])
            supporters_data = np.zeros((len(results[timing]['all_histories']), num_steps))
            
            # Collect data from all trials
            for k, hist in enumerate(results[timing]['all_histories']):
                if len(hist) == num_steps:  # Ensure same length
                    supporters_data[k] = [h[SUPPORTER] for h in hist]
            
            # Calculate means and standard deviations
            supporters_mean = np.mean(supporters_data, axis=0)
            supporters_std = np.std(supporters_data, axis=0)
            
            steps = range(num_steps)
            
            # Plot mean line
            ax1.plot(steps, supporters_mean, '-', color=colors[timing], 
                    linewidth=2.5, label=f'{timing}')
            
            # Add confidence interval (mean ± 1 std)
            ax1.fill_between(steps, 
                           supporters_mean - supporters_std, 
                           supporters_mean + supporters_std, 
                           color=colors[timing], alpha=0.2)
            
            # Mark intervention period if known
            shock_start = None
            shock_duration = None
            
            if timing == 'Early Campaign':
                shock_start = 10
                shock_duration = 20
            elif timing == 'Late Campaign':
                shock_start = 50
                shock_duration = 20
                
            if shock_start is not None and shock_duration is not None:
                ax1.axvspan(shock_start, shock_start + shock_duration, 
                          color=colors[timing], alpha=0.15, edgecolor=colors[timing],
                          linewidth=1, zorder=1)
                
                # Add label for intervention period
                ax1.text(shock_start + shock_duration/2, 0.98, timing,
                       ha='center', va='top', color=colors[timing], fontsize=10,
                       transform=ax1.get_xaxis_transform(),
                       bbox=dict(facecolor='white', alpha=0.7, pad=2))
        
        # Fallback to using history if all_histories is not available
        elif 'history' in results[timing]:
            history = results[timing]['history']
            supporters = np.array([h[SUPPORTER] for h in history])
            steps = range(len(history))
            
            # Plot with consistent colors
            ax1.plot(steps, supporters, '-', color=colors[timing], linewidth=2.5, label=timing)
    
    # Add styling
    ax1.set_title('Impact of Intervention Timing on Opinion Evolution', fontsize=16, pad=15)
    ax1.set_xlabel('Time Step', fontsize=14)
    ax1.set_ylabel('Supporter Proportion', fontsize=14)
    ax1.set_ylim(0, 1)
    ax1.grid(True, alpha=0.3)
    ax1.legend(title='Timing Strategy', fontsize=12, loc='upper left')
    
    # Bottom left: Bar chart of final supporter proportions
    ax2 = fig.add_subplot(gs[1, 0])
    
    # Calculate means and standard errors for bar chart
    supporter_means = [np.mean(results[timing]['supporter_final']) for timing in timing_labels]
    supporter_stds = [np.std(results[timing]['supporter_final']) for timing in timing_labels]
    
    # Plot bars with error bars
    bars = ax2.bar(range(len(timing_labels)), supporter_means, yerr=supporter_stds, 
                  capsize=10, color=[colors[timing] for timing in timing_labels], 
                  edgecolor='black', linewidth=1)
    
    # Add data labels on top of bars
    for i, bar in enumerate(bars):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2, height + supporter_stds[i] + 0.01,
                f'{supporter_means[i]:.3f}', ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # Add styling
    ax2.set_title('Final Supporter Proportion', fontsize=14)
    ax2.set_ylabel('Proportion', fontsize=12)
    ax2.set_xticks(range(len(timing_labels)))
    ax2.set_xticklabels(timing_labels, fontsize=10)
    ax2.set_ylim(0, 1)
    ax2.grid(axis='y', alpha=0.3, linestyle='--')
    
    # Bottom right: Campaign effectiveness metrics
    ax3 = fig.add_subplot(gs[1, 1])
    
    # Calculate gain metrics (final - initial)
    initial_supporter = 0.3  # From initial_states proportion in run_timing_experiment
    gain_means = [mean - initial_supporter for mean in supporter_means]
    gain_stds = supporter_stds  # Same std for the difference
    
    # Calculate efficiency metrics (gain per unit of intervention)
    efficiency_means = [gain / 20 for gain in gain_means]  # Assuming 20 time steps of intervention
    efficiency_stds = [std / 20 for std in gain_stds]
    
    # Plot bars with error bars - different metric
    ax3.bar([0, 1], gain_means, yerr=gain_stds, 
           capsize=10, color=['#aec7e8', '#ff9896'], 
           edgecolor='black', linewidth=1, label='Absolute Gain')
    
    # Add second set of bars for efficiency
    ax3_2 = ax3.twinx()
    ax3_2.bar([2, 3], efficiency_means, yerr=efficiency_stds,
             capsize=10, color=['#c5dbef', '#ffbcba'], 
             edgecolor='black', linewidth=1, label='Efficiency')
    
    # Add styling
    ax3.set_title('Campaign Effectiveness', fontsize=14)
    ax3.set_ylabel('Absolute Gain', fontsize=12)
    ax3_2.set_ylabel('Gain per Time Step', fontsize=12)
    ax3.set_xticks([0, 1, 2, 3])
    ax3.set_xticklabels(['Early\nGain', 'Late\nGain', 'Early\nEfficiency', 'Late\nEfficiency'], 
                      fontsize=9)
    ax3.grid(axis='y', alpha=0.3, linestyle='--')
    
    # Create legend with both metrics
    lines1, labels1 = ax3.get_legend_handles_labels()
    lines2, labels2 = ax3_2.get_legend_handles_labels()
    ax3_2.legend(lines1 + lines2, labels1 + labels2, loc='upper right', fontsize=10)
    
    # Add a network science-oriented header
    plt.suptitle('Temporal Dynamics of Opinion Spread in Scale-Free Networks', 
                fontsize=18, y=0.98)
    plt.tight_layout()
    fig.subplots_adjust(top=0.92, hspace=0.35)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    # Save figure
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    print(f"Figure saved to {save_path}")
    
    return fig

def plot_cross_network_results(results, save_path="figures/cross_network_comparison.pdf"):
    """
    Plot comprehensive comparison of intervention strategies across network types.
    Creates publication-quality visualizations with proper statistical rigor.
    
    Parameters:
    -----------
    results : dict
        Results dictionary from run_cross_network_intervention_comparison()
    save_path : str
        Path to save the figure
        
    Returns:
    --------
    matplotlib.figure.Figure
        Figure containing the plots
    """
    # Set seaborn style for publication quality
    import seaborn as sns
    sns.set_theme(style="whitegrid", context="paper")
    
    network_types = list(results.keys())
    if not network_types:
        return None
        
    # Get intervention strategies from first network type
    interventions = list(results[network_types[0]].keys())
    
    # Create figure with subplots
    fig = plt.figure(figsize=(15, 10))
    gs = GridSpec(2, 2, figure=fig)
    
    # Top left: Bar chart of final supporter proportions across networks and interventions
    ax1 = fig.add_subplot(gs[0, 0])
    
    # Prepare data for grouped bar chart
    bar_positions = []
    bar_heights = []
    bar_errors = []
    bar_colors = []
    bar_labels = []
    
    # Color palette for networks
    network_palette = sns.color_palette("husl", len(network_types))
    network_colors = {network: network_palette[i] for i, network in enumerate(network_types)}
    
    # X positions for groups of bars
    group_width = 0.8
    bar_width = group_width / len(network_types)
    offsets = np.linspace(-group_width/2 + bar_width/2, group_width/2 - bar_width/2, len(network_types))
    
    for i, intervention in enumerate(interventions):
        for j, network in enumerate(network_types):
            # Calculate mean and std
            supporter_mean = np.mean(results[network][intervention]['supporter_final'])
            supporter_std = np.std(results[network][intervention]['supporter_final'])
            
            # Store position and value
            bar_positions.append(i + offsets[j])
            bar_heights.append(supporter_mean)
            bar_errors.append(supporter_std)
            bar_colors.append(network_colors[network])
            bar_labels.append(f"{network}-{intervention}")
    
    # Plot bars with error bars
    bars = ax1.bar(bar_positions, bar_heights, yerr=bar_errors, 
                  width=bar_width, color=bar_colors, 
                  edgecolor='black', linewidth=1, capsize=3)
    
    # Add styling
    ax1.set_title('Final Supporter Proportion by Intervention and Network', fontsize=14)
    ax1.set_ylabel('Supporter Proportion', fontsize=12)
    ax1.set_xticks(range(len(interventions)))
    ax1.set_xticklabels(interventions, fontsize=10, rotation=45, ha='right')
    ax1.set_ylim(0, 1)
    
    # Custom legend for network types
    network_handles = [plt.Rectangle((0,0),1,1, color=network_colors[nt]) for nt in network_types]
    ax1.legend(network_handles, network_types, title='Network Type', 
              fontsize=9, loc='upper left')
    
    # Top right: Line chart showing opinion evolution over time for different networks
    # Focus on one intervention strategy to keep it clear
    focus_intervention = 'High-Degree Targeting'
    ax2 = fig.add_subplot(gs[0, 1])
    
    for i, network in enumerate(network_types):
        # Check if we have multiple histories for confidence intervals
        if 'all_histories' in results[network][focus_intervention] and len(results[network][focus_intervention]['all_histories']) > 0:
            # Calculate confidence intervals
            histories = results[network][focus_intervention]['all_histories']
            num_steps = len(histories[0])
            supporters_data = np.zeros((len(histories), num_steps))
            
            # Collect data from all trials
            for k, hist in enumerate(histories):
                if len(hist) == num_steps:
                    supporters_data[k] = [h[SUPPORTER] for h in hist]
            
            # Calculate means and standard deviations
            supporters_mean = np.mean(supporters_data, axis=0)
            supporters_std = np.std(supporters_data, axis=0)
            
            steps = range(num_steps)
            
            # Plot mean line
            ax2.plot(steps, supporters_mean, '-', color=network_colors[network],
                    linewidth=2, label=network)
            
            # Add confidence interval
            ax2.fill_between(steps, 
                           supporters_mean - supporters_std, 
                           supporters_mean + supporters_std, 
                           color=network_colors[network], alpha=0.2)
        else:
            # Fallback to representative history
            history = results[network][focus_intervention]['supporter_history']
            steps = range(len(history))
            ax2.plot(steps, history, '-', color=network_colors[network],
                    linewidth=2, label=network)
    
    # Highlight intervention period
    ax2.axvspan(30, 30 + shock_duration, color='gray', alpha=0.2, 
               label='Intervention Period')
    
    # Add styling
    ax2.set_title(f'Opinion Evolution: {focus_intervention} Strategy', fontsize=14)
    ax2.set_xlabel('Time Step', fontsize=12)
    ax2.set_ylabel('Supporter Proportion', fontsize=12)
    ax2.set_ylim(0, 1)
    ax2.legend(title='Network Type', fontsize=9, loc='upper left')
    
    # Bottom left: Network metrics comparison
    ax3 = fig.add_subplot(gs[1, 0])
    
    # Prepare data for clustering coefficient and homophily change
    clustering_changes = []
    homophily_changes = []
    network_labels = []
    colors = []
    
    for network in network_types:
        for intervention in interventions:
            if len(results[network][intervention]['network_metrics']) > 0:
                metrics = results[network][intervention]['network_metrics']
                
                # Calculate average changes in metrics
                clustering_change = np.mean([m['final']['clustering'] - m['initial']['clustering'] for m in metrics])
                homophily_change = np.mean([m['final']['homophily'] - m['initial']['homophily'] for m in metrics])
                
                clustering_changes.append(clustering_change)
                homophily_changes.append(homophily_change)
                network_labels.append(f"{network}-{intervention}")
                colors.append(network_colors[network])
    
    # Create scatter plot
    ax3.scatter(homophily_changes, clustering_changes, c=colors, s=100, alpha=0.7, edgecolor='black')
    
    # Label points
    for i, label in enumerate(network_labels):
        ax3.annotate(label, (homophily_changes[i], clustering_changes[i]),
                    fontsize=8, ha='right', va='bottom',
                    xytext=(5, 5), textcoords='offset points')
    
    # Add styling
    ax3.set_title('Network Structural Changes by Intervention', fontsize=14)
    ax3.set_xlabel('Change in Opinion Homophily', fontsize=12)
    ax3.set_ylabel('Change in Clustering Coefficient', fontsize=12)
    ax3.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    ax3.axvline(x=0, color='black', linestyle='--', alpha=0.5)
    ax3.grid(True, alpha=0.3)
    
    # Bottom right: Bar chart comparing intervention effectiveness
    ax4 = fig.add_subplot(gs[1, 1])
    
    # Calculate intervention effectiveness across network types
    effect_data = []
    
    for intervention in interventions:
        # Skip No Intervention as baseline
        if intervention == 'No Intervention':
            continue
            
        for network in network_types:
            # Calculate effectiveness as difference from No Intervention
            baseline = np.mean(results[network]['No Intervention']['supporter_final'])
            with_intervention = np.mean(results[network][intervention]['supporter_final'])
            effectiveness = with_intervention - baseline
            
            # Calculate standard error
            baseline_std = np.std(results[network]['No Intervention']['supporter_final'])
            intervention_std = np.std(results[network][intervention]['supporter_final'])
            # Combined standard error (approximation)
            std_error = np.sqrt(baseline_std**2 + intervention_std**2) / np.sqrt(num_trials)
            
            effect_data.append({
                'Network': network,
                'Intervention': intervention,
                'Effectiveness': effectiveness,
                'StdError': std_error
            })
    
    # Convert to DataFrame for seaborn
    import pandas as pd
    df = pd.DataFrame(effect_data)
    
    # Plot with seaborn
    sns.barplot(x='Intervention', y='Effectiveness', hue='Network', data=df, 
               palette=network_colors, errorbar='sd', ax=ax4)
    
    # Add styling
    ax4.set_title('Intervention Effectiveness by Network Type', fontsize=14)
    ax4.set_xlabel('Intervention Strategy', fontsize=12)
    ax4.set_ylabel('Effectiveness (vs. No Intervention)', fontsize=12)
    ax4.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    plt.setp(ax4.get_xticklabels(), rotation=45, ha='right')
    
    # Add a network science-oriented header
    plt.suptitle('Network Topology Effects on Opinion Intervention Strategies', 
                fontsize=18, y=0.98)
    plt.tight_layout()
    fig.subplots_adjust(top=0.92, wspace=0.3, hspace=0.3)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    # Save figure
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    print(f"Figure saved to {save_path}")
    
    return fig

def plot_enhanced_network_metrics(metrics_history, title=None, shock_period=None):
    """
    Create a publication-quality plot of network metrics over time.
    
    Parameters:
    -----------
    metrics_history : dict
        Dictionary with metrics over time from track_network_metrics_over_time()
    title : str or None
        Main title for the figure
    shock_period : tuple or None
        (start, end) of shock period to highlight
    
    Returns:
    --------
    matplotlib.figure.Figure
        Figure containing the plots
    """
    fig = plt.figure(figsize=(15, 12))
    gs = GridSpec(3, 3, figure=fig)
    
    # Plot 1: Homophily
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.plot(metrics_history['homophily'], color='#ff7f0e', linewidth=2.5)
    ax1.set_ylabel('Homophily')
    ax1.set_title('Opinion Similarity in Connections')
    
    # Plot 2: Polarization
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.plot(metrics_history['polarization'], color='#d62728', linewidth=2.5)
    ax2.set_ylabel('Polarization')
    ax2.set_title('Supporter-Opposition Connections')
    
    # Plot 3: Segregation Index
    ax3 = fig.add_subplot(gs[0, 2])
    ax3.plot(metrics_history['segregation_index'], color='#9467bd', linewidth=2.5)
    ax3.set_ylabel('Segregation Index')
    ax3.set_title('Opinion Clustering vs Random')
    
    # Plot 4: Opinion Clusters
    ax4 = fig.add_subplot(gs[1, :])
    ax4.plot(metrics_history['supporter_clusters'], color=SUPPORTER_COLOR, linewidth=2.5, label='Supporter Clusters')
    ax4.plot(metrics_history['undecided_clusters'], color=UNDECIDED_COLOR, linewidth=2.5, label='Undecided Clusters')
    ax4.plot(metrics_history['opposition_clusters'], color=OPPOSITION_COLOR, linewidth=2.5, label='Opposition Clusters')
    ax4.set_ylabel('Number of Clusters')
    ax4.set_title('Opinion Fragmentation Over Time')
    ax4.legend(loc='upper right')
    
    # Plot 5: Influence by Opinion
    ax5 = fig.add_subplot(gs[2, :])
    ax5.plot(metrics_history['top_supporter_influence'], color=SUPPORTER_COLOR, linewidth=2.5, label='Supporter Influence')
    ax5.plot(metrics_history['top_undecided_influence'], color=UNDECIDED_COLOR, linewidth=2.5, label='Undecided Influence')
    ax5.plot(metrics_history['top_opposition_influence'], color=OPPOSITION_COLOR, linewidth=2.5, label='Opposition Influence')
    ax5.set_ylabel('Influence Score')
    ax5.set_xlabel('Time Step')
    ax5.set_title('Top Influencer Scores by Opinion')
    ax5.legend(loc='upper right')
    
    # Add shock period highlighting if provided
    if shock_period:
        for ax in [ax1, ax2, ax3, ax4, ax5]:
            ax.axvspan(shock_period[0], shock_period[1], alpha=0.2, color='gray')
            if ax == ax4:  # Only add label on one plot
                ax.text((shock_period[0] + shock_period[1])/2, ax.get_ylim()[1]*0.9, 'Intervention', 
                        horizontalalignment='center', fontsize=12, bbox=dict(facecolor='white', alpha=0.7))
    
    # Add overall title if provided
    if title:
        fig.suptitle(title, fontsize=18, y=0.98)
    
    plt.tight_layout()
    fig.subplots_adjust(top=0.9 if title else 0.95)
    
    return fig






